In [ ]:
# Load autoreload extension for changes in `utils.py` to take effect even without a kernel restart + enable ipywidgets
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import os
import pathlib
import xarray as xr
import matplotlib.colors as mcolors
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from utils import (
    print_dataset_variables_summary,
    truncate_cmap,
    mark_and_store_points_onclick,
    plot_figures,
    plot_cross_section_between_two_points, plot_cross_section_across_dim, estimate_topography_from_dataset
)

## Load data

In [ ]:
# Specify the path to the directory containing .nc files
DATA_DIR_PATH = "kenda-ch1-eps_MDR_3D_d_2024112110"
data_dir = pathlib.Path(DATA_DIR_PATH)

In [ ]:
ds_list = []
for filename in sorted(os.listdir(data_dir)):
    ds = xr.open_dataset(data_dir / filename)
    assert len(ds["time"]) == 1
    ds = ds.isel(time=0)
    ds_list.append(ds)

print_dataset_variables_summary(ds_list[0])

## Plot topography
The simulated data will be analyzed through vertical cross-sections in the 3D model domain over Switzerland.

To define the transect for cross-sections, **choose two points** on the topography plot.

In [ ]:
coord_label = widgets.Label(value="Click two points...")
display(coord_label)

fig, ax = plt.subplots(figsize=(12, 6))
cmap_terrain = truncate_cmap(plt.cm.terrain, start=0.25, stop=0.75)
topography = estimate_topography_from_dataset(ds_list[0])
topography.plot(x="x_1", y="y_1", cmap=cmap_terrain, ax=ax)
plt.title("Topography over KENDA-CH1-EPS domain")

selected_points = []
markers = []           # store marker artists
line = None            # store line artist

def onclick(event):
    global selected_points, markers, line

    if event.inaxes != ax:
        return

    selected_points, markers, line = mark_and_store_points_onclick(event, coord_label, ds_list[0], markers, line, selected_points)
    fig.canvas.draw_idle()

fig.canvas.mpl_connect("button_press_event", onclick)
plt.show()

## Plot vertical cross-sections

In [ ]:
figures = []

for ds in ds_list:
    fig = plot_cross_section_between_two_points(ds, selected_points[0], selected_points[1], var="RELHUM", var_contour="THETA", plot_kwargs=dict(cmap="YlGn", norm=mcolors.Normalize(vmin=0, vmax=100)))
    figures.append(fig)

plot_figures(figures)

In [ ]:
figures = []

for ds in ds_list:
    fig = plot_cross_section_between_two_points(ds, selected_points[0], selected_points[1], var="FF", var_contour="THETA", plot_kwargs=dict(norm=mcolors.Normalize(vmin=0, vmax=65)))
    figures.append(fig)

plot_figures(figures)

## Analyze mid-upper level stability

In [ ]:
figures = []

for ds in ds_list:
    fig = plot_cross_section_across_dim(ds, "z_1", 5000, var="v_on_z", plot_kwargs=dict(cmap="seismic", norm=mcolors.Normalize(vmin=-3, vmax=3)))
    figures.append(fig)

plot_figures(figures)

In [ ]:
figures = []

for ds in ds_list:
    fig = plot_cross_section_between_two_points(ds, selected_points[0], selected_points[1], var="v_on_z", plot_kwargs=dict(cmap="seismic", norm=mcolors.Normalize(vmin=-3, vmax=3)))
    figures.append(fig)

plot_figures(figures)

In [ ]:
plt.close("all")